In [1]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [76]:
subj = subject.Subject & 'subject_nickname="IBL_46"'
# get fit parameters by date
fit_pars = (behavior.BehavioralSummaryByDate.PsychResults * subj).proj(
    'session_date', 'prob_left','threshold', 'bias', 'lapse_low', 'lapse_high').fetch(as_dict=True)
fit_pars = pd.DataFrame(fit_pars)

In [69]:
par_names = ['threshold', 'bias', 'lapse_low', 'lapse_high']

pars = dict()
for par_name in par_names:
    pars[par_name] = []

prob_lefts = fit_pars['prob_left'].unique()
    
for prob_left in prob_lefts:
    prob_left_filter = fit_pars['prob_left']==prob_left
    if prob_left == 0.2:
        dot_color = 'orange'
    elif prob_left == 0.5:
        dot_color = 'black'
    elif prob_left == 0.8:
        dot_color = 'cornflowerblue'
    else:
        dot_color = 'gray'
    
    fit_pars_sub = fit_pars[prob_left_filter]
    
    for par_name in par_names:
        pars[par_name].append(
            go.Scatter(      
                x=[t.strftime('%Y-%m-%d') for t in fit_pars_sub['session_date'].tolist()],
                y=fit_pars_sub[par_name].tolist(),
                mode='markers',
                marker=dict(
                    size=5,
                    color=dot_color),
                name=f'p_left = {prob_left}')
        )


In [73]:
par_title_names = ['Threshold', 'Bias', 'Lapse\ low', 'Lapse\ high']
par_symbols = ['sigma', 'mu', 'gamma', 'lambda']
ranges = [[-5, 105], [-105, 105], [-0.02, 1.02], [-0.02, 1.02]]
for ipar, par_name in enumerate(par_names):
    layout = go.Layout(
        width=500,
        height=400,
        title='', 
        xaxis={'title':'Date'}, 
        yaxis=dict(
            title='${}\ (\\{})$'.format(
            par_title_names[ipar], par_symbols[ipar]),
            range = ranges[ipar]
        ),
    )
    fig = go.Figure(data=pars[par_name], layout=layout)
    plotly.offline.iplot(fig)
    f = open('{}.json'.format(par_name), 'w')
    s = json.dumps(fig.to_plotly_json())
    f.write(s)
    f.close()